# Problem Set \#3

## Eren Bilen

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.optimize import minimize
import scipy.stats as stats
import statsmodels.formula.api as sm
import time

## Reading in the data

In [2]:
dataset = pd.read_stata('PS3_data.dta')

## Cleaning the data and defining variables

In [3]:
# Dropping NaN observations
dataset = dataset.dropna(how='any', subset=['hannhrs', 'hlabinc','hyrsed','hrace'])

In [4]:
# Choosing only male heads
dataset = dataset[dataset.hsex == 1]

In [5]:
# Dropping individuals who worked 0 hours
dataset = dataset[dataset.hannhrs!=0]

In [6]:
# Defining hourly wage
dataset['hourlywage'] = dataset.hlabinc / dataset.hannhrs

In [7]:
# Choosing the age group
dataset = dataset[(dataset.age>=25) & (dataset.age<=60)]

In [8]:
# Choosing individuals who earn more than $7 per hour
dataset = dataset[dataset.hourlywage>7]

In [9]:
# Taking log of hourly wage
dataset['loghourlywage'] = np.log(dataset.hourlywage)

In [10]:
# Defining race dummies
dataset['white'] = (dataset['hrace'] == 1).astype(int)
dataset['black'] = (dataset['hrace'] == 2).astype(int)
dataset['hispanic'] = (dataset['hrace'] == 5).astype(int)
dataset['other'] = ((dataset['hrace'] == 3) | (dataset['hrace']==4) | (dataset['hrace']==6) | (dataset['hrace']==7)).astype(int)

In [11]:
# Also note that there are no Hispanic individuals in the data hence this 
# variable will be disregarded so that the data matrix X remains non-singular.
dataset['hispanic'].unique()

array([0], dtype=int64)

In [12]:
# Creating dataframes for each year
dataset1971 = dataset[dataset.year == 1971]
dataset1980 = dataset[dataset.year == 1980]
dataset1990 = dataset[dataset.year == 1990]
dataset2000 = dataset[dataset.year == 2000]

## For 1971

In [13]:
import numpy
b1 = np.array(dataset1971['hyrsed']).astype('float')
b2 = np.array(dataset1971['black']).astype('float')
b3 = np.array(dataset1971['hispanic']).astype('float')
b4 = np.array(dataset1971['other']).astype('float')
b5 = np.array(dataset1971['age']).astype('float')
y = np.array(dataset1971['loghourlywage']).astype('float')

# Creating matrix X, which will be useful soon

nrow = b1.shape[0]
intercept = np.ones((nrow,1))
X=numpy.column_stack((intercept,b1,b2,b4,b5))

## Maximum Likelihood for 1971

In [14]:
# Define a function that returns the negative log likelihood where "parameters" is a list for initial parameter values

def LL(parameters):
    
    beta0 = parameters[0]
    beta1 = parameters[1]
    beta2 = parameters[2]
    beta4 = parameters[3]
    beta5 = parameters[4]
    sd = parameters[5]
    
    # Creating a Beta vector that consists of the coefficients
    Beta = [beta0, beta1, beta2, beta4, beta5]

    # Calculating the predicted values using initial guesses for coefficients
    yPred = np.dot(X, Beta)

    # Calculating the negative log-likelihood assuming the values are normally distributed around the mean (yPred)
    # and standard deviation, sd
    neglogLik = -np.sum(stats.norm.logpdf(y, loc=yPred, scale=sd))

    return(neglogLik)

In [15]:
# This is a list for our initial parameter guesses (beta0, beta1, beta2, beta4, beta5, sd)    
initGuess = [.1, .1, .1, .1, .1, .1 ]

In [16]:
# Run the minimization process
results1971 = minimize(LL, initGuess, method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

In [17]:
# Results
print("Coefficient estimates for 1971: ", results1971.x)

Coefficient estimates for 1971:  [ 1.5509638   0.06687878 -0.16388756  0.03068778  0.01439146  0.41009898]


These are the coefficients that maximize the log likelihood function. The order of the output is 

[$a, b_{educ}, b_{black}, b_{other}, b_{age}, sd$]

As will be demonstrated below, these coefficients are almost identical to the OLS coefficients. More information on the optimization process:

In [18]:
results1971

 final_simplex: (array([[ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898],
       [ 1.5509638 ,  0.06687878, -0.16388756,  0.03068778,  0.01439146,
         0.41009898]]), array([ 728.06286706,  728.06286706,  728.06286706,  728.06286706,
        728.06286706,  728.06286706,  728.06286706]))
           fun: 728.06286706384549
       message: 'Optimization terminated successfully.'
          nfev: 1708
           nit: 1033
        status: 0
       success: True
             x: array([

## Checking work using linear regression

In [19]:
OLSresults1971 = sm.ols(formula="y ~ b1 + b2 + b3 + b4 + b5", data=dataset1971).fit()
print(OLSresults1971.params)

Intercept    1.550964e+00
b1           6.687878e-02
b2          -1.638876e-01
b3           2.938373e-17
b4           3.068777e-02
b5           1.439146e-02
dtype: float64


These coefficients are almost identical to the coefficients we get from MLE.

## Alternatively, we could run OLS manually:

In [20]:
OLSresults_manual = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,y))
OLSresults_manual

array([ 1.55096381,  0.06687878, -0.16388756,  0.03068777,  0.01439146])

We get the same coefficients.

## MLE for 1980

In [21]:
b1 = np.array(dataset1980['hyrsed']).astype('float')
b2 = np.array(dataset1980['black']).astype('float')
b3 = np.array(dataset1980['hispanic']).astype('float')
b4 = np.array(dataset1980['other']).astype('float')
b5 = np.array(dataset1980['age']).astype('float')
y = np.array(dataset1980['loghourlywage']).astype('float')

nrow = b1.shape[0]
intercept = np.ones((nrow,1))
X=numpy.column_stack((intercept,b1,b2,b4,b5))

# Run the minimization process
results1980 = minimize(LL, initGuess, method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

# Results
print("Coefficient estimates for 1980: ", results1980.x)

Coefficient estimates for 1980:  [ 1.61308833  0.06755585 -0.10273614  0.01351126  0.01269854  0.44924262]


## MLE for 1990

In [22]:
b1 = np.array(dataset1990['hyrsed']).astype('float')
b2 = np.array(dataset1990['black']).astype('float')
b3 = np.array(dataset1990['hispanic']).astype('float')
b4 = np.array(dataset1990['other']).astype('float')
b5 = np.array(dataset1990['age']).astype('float')
y = np.array(dataset1990['loghourlywage']).astype('float')

nrow = b1.shape[0]
intercept = np.ones((nrow,1))
X=numpy.column_stack((intercept,b1,b2,b4,b5))

# Run the minimization process
results1990 = minimize(LL, initGuess, method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

# Results
print("Coefficient estimates for 1990: ", results1990.x)

Coefficient estimates for 1990:  [ 1.11858497  0.09755808 -0.17202431 -0.05971251  0.0134656   0.48359944]


## MLE for 2000

In [23]:
b1 = np.array(dataset2000['hyrsed']).astype('float')
b2 = np.array(dataset2000['black']).astype('float')
b3 = np.array(dataset2000['hispanic']).astype('float')
b4 = np.array(dataset2000['other']).astype('float')
b5 = np.array(dataset2000['age']).astype('float')
y = np.array(dataset2000['loghourlywage']).astype('float')

nrow = b1.shape[0]
intercept = np.ones((nrow,1))
X=numpy.column_stack((intercept,b1,b2,b4,b5))

# Run the minimization process
results2000 = minimize(LL, initGuess, method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

# Results
print("Coefficient estimates for 2000: ", results2000.x)

Coefficient estimates for 2000:  [ 1.16169082  0.10915436 -0.24604485 -0.06073246  0.01099353  0.53955729]


## Comparing years

In [24]:
print("Coefficient estimates for 1971: ", results1971.x)
print("Coefficient estimates for 1980: ", results1980.x)
print("Coefficient estimates for 1990: ", results1990.x)
print("Coefficient estimates for 2000: ", results2000.x)

Coefficient estimates for 1971:  [ 1.5509638   0.06687878 -0.16388756  0.03068778  0.01439146  0.41009898]
Coefficient estimates for 1980:  [ 1.61308833  0.06755585 -0.10273614  0.01351126  0.01269854  0.44924262]
Coefficient estimates for 1990:  [ 1.11858497  0.09755808 -0.17202431 -0.05971251  0.0134656   0.48359944]
Coefficient estimates for 2000:  [ 1.16169082  0.10915436 -0.24604485 -0.06073246  0.01099353  0.53955729]


Once again, the order of the output is

[$a, b_{educ}, b_{black}, b_{other}, b_{age}, sd$]

We can see from the second column that, the coefficient estimate on education was about 0.066 in 1971. This would translate to an interpretation that, per additional year in education results in a 6.6% increase in hourly wage for an individual. The magnitude of this coefficient was about the same in 1980. Yet, in 1990, the magnitide goes up to 0.0975. Now, an additional year in education translates to a 9.75% increase in wage. The magnitude further increases to 10.9% in 2000. All in all, we see from this analysis that there is a positive association between education and wages. The magnitude of this association is higher in 1990-2000 period than it is for the 1971-1980 period.